# 맛집 추천  시스템 

## 데이터 전처리 작업

In [1]:
import pandas as pd

#### 데이터 불러오기

In [2]:
data = pd.read_csv('./data/네이버맵음식점평점_최종.csv',index_col=0)

In [32]:
data.head()

,name,upjong,address,category,rating,write_id
0,설레임,한식,"제주특별자치도 제주시 애월읍 일주서로 5999-1, 2층",애월읍,5.0,5dee59b58f87a842bc875a3a
4,설레임,한식,"제주특별자치도 제주시 애월읍 일주서로 5999-1, 2층",애월읍,5.0,5c381b1c5745ec219264a906
8,설레임,한식,"제주특별자치도 제주시 애월읍 일주서로 5999-1, 2층",애월읍,5.0,5df37ace8f87a842bc889e8c
12,설레임,한식,"제주특별자치도 제주시 애월읍 일주서로 5999-1, 2층",애월읍,5.0,5d4fac00a2b372d9cf8c57be
16,설레임,한식,"제주특별자치도 제주시 애월읍 일주서로 5999-1, 2층",애월읍,5.0,5f05c9d4699d39675d9e8551


#### 리뷰 갯수

In [85]:
data.value_counts('name')

name
고집돌우럭 중문점     5845
문개항아리조천본점     5188
설레임           5042
델문도           3183
제주순풍해장국함덕점    2961
              ... 
남촌               1
펠롱               1
비비큐치킨추자점         1
땅땅치킨삼도점          1
남이네육개장           1
Length: 7311, dtype: int64

#### 사용자별 식당 평점

In [58]:
star_data = data.pivot_table(index='write_id',columns='name',values='rating')

In [4]:
# # 리뷰 개수 10개 이상 작성한 id만 추출
# star_data_10 = star_data[star_data.count(axis=1)>=10]
# # 리뷰 개수 20개 이상 작성한 id만 추출
# star_data_20 = star_data[star_data.count(axis=1)>=20]

# #  리뷰가 없는 식당 삭제
# star_data_10_T = star_data_10.T[star_data_10.T.sum(axis=1)>0]
# star_data_10 = star_data_10_T.T

# # #NaN값을 -1로 변경 ( 평점을 계산할 때 양수값만 처리하면 됌)
# star_data_10.fillna(-1,inplace=True)

# # # 아이디값 숫자로 변환
# star_data_10.reset_index(drop=True,inplace=True)

# star_data_10.to_csv('data/star_data_10.csv')
star_data_10 = pd.read_csv('data/star_data_10.csv',index_col=0)

In [5]:
# # # 리뷰 개수 20개 이상 작성한 id만 추출
# star_data_20 = star_data[star_data.count(axis=1)>=20]

# #  리뷰가 없는 식당 삭제
# star_data_20_T = star_data_20.T[star_data_20.T.sum(axis=1)>0]
# star_data_20 = star_data_20_T.T

# # #NaN값을 -1로 변경 ( 평점을 계산할 때 양수값만 처리하면 됌)
# star_data_20.fillna(-1,inplace=True)

# # # 아이디값 숫자로 변환
# star_data_20.reset_index(drop=True,inplace=True)

# star_data_20.to_csv('data/star_data_20.csv')
star_data_20 = pd.read_csv('data/star_data_20.csv',index_col=0)

#### 평균 별점

In [13]:
avg_star = data.pivot_table('rating',index='name',aggfunc='mean')

In [8]:
avg_star.head()

,rating
name,
(주)삼다도횟집,4.350000
09학번,5.000000
1.5커피,5.000000
11시11분,4.684211
168테우카페,4.650000


## 맛집 추천 시스템 함수화(가중 평점 기반)

#### 전처리 및 가중치 함수화

In [14]:
def make_review_data(data, area, percentile=0.6):

    cnt_review = pd.DataFrame(data.value_counts('name'))
    avg_star = data.pivot_table('rating',index='name',aggfunc='mean')

    # 평점/리뷰갯수 정보를 가지는 temp 변수 생성
    temp = avg_star.merge(cnt_review,left_index=True,right_index=True,how='left')
    temp.columns = ['rating','num']
    
    temp= temp.fillna(0)
    
    temp

    
    m = temp['num'].quantile(percentile)
    C = temp['rating'].mean()

    
    #가충치 함수를 람다함수로 처리
    temp['weighted_rating'] = temp.apply((lambda x: (x['num']/(x['num']+m)*x['rating'])+(m/(m+x['num'])*C)), axis=1)
    
    
    ## 주소 부분
    # 음식점 이름 + 주소를 기준으로 그룹화 시켜서 
    address =pd.DataFrame(data.groupby(['name'])['address'].first())
    
    # 주소 부분 병합
    temp=temp.merge(address,left_index=True,right_index=True)
    
    
    ## 지역부분
    # 음식점 이름 + 지역을 기준으로 그룹화 시켜서 
    name_area =pd.DataFrame(data.groupby(['name'])['category'].first())
    
    # 평점/리뷰갯수에 음식점 이름 + 지역 병합
    temp=temp.merge(name_area,left_index=True,right_index=True)
    
    
    # 사용자 위치에 해당되는 정보만 정리
    temp=temp[temp['category']==area]
    
    temp
    # 정렬후 반환
    return temp[['rating','weighted_rating','num','category','address']].sort_values('weighted_rating', ascending=False)[:10]

## 실제 구현 부분

In [15]:
# 해당 지점을 DB랑 연결
data = pd.read_csv('./data/네이버맵음식점평점_최종.csv')

if __name__=='__main__':
    print('사용자 위치를 입력해주세요')
    print()
    input_data_area=input()
    # 식당 순위 info에 저장
    info = make_review_data(data, input_data_area)
    
    info.reset_index(inplace=True)
    
    print()
    print()
    # 식당 순위 출력
    for i in range(10):
        print(i+1,'순위')
        print('음식점 이름 : ', info['name'][i], '\t평균 평점 : ', round(info['weighted_rating'][i],2))
        print()
        print()

사용자 위치를 입력해주세요

제주시


1 순위
음식점 이름 :  연화차 	평균 평점 :  4.96


2 순위
음식점 이름 :  제주공항본점공항뚝배기 	평균 평점 :  4.94


3 순위
음식점 이름 :  더팔당제주 	평균 평점 :  4.92


4 순위
음식점 이름 :  램스키친 	평균 평점 :  4.92


5 순위
음식점 이름 :  덤장돌삼겹 	평균 평점 :  4.91


6 순위
음식점 이름 :  우돈향 	평균 평점 :  4.91


7 순위
음식점 이름 :  반디파스타 	평균 평점 :  4.91


8 순위
음식점 이름 :  돈마니 	평균 평점 :  4.9


9 순위
음식점 이름 :  팔각촌 	평균 평점 :  4.89


10 순위
음식점 이름 :  닥그네할망 	평균 평점 :  4.88




In [11]:
info

,name,rating,weighted_rating,num,category,address
0,쉬다가게,4.960648,4.923976,216,애월읍,"제주특별자치도 제주시 애월읍 구엄길 66, 1층"
1,호커센터,4.920394,4.906717,559,애월읍,제주특별자치도 제주시 애월읍 애월로11길 25-2
2,제주기와,4.913000,4.897989,500,애월읍,"제주특별자치도 제주시 애월읍 광령남4길 45-1, 1층"
3,포구횟집,4.898798,4.884199,499,애월읍,제주특별자치도 제주시 애월읍 가문동길 43
4,누렁소앤도새기,4.948980,4.875987,98,애월읍,"제주특별자치도 제주시 애월읍 광령남6길 14, 1층"
5,카페앤틱애월,4.975806,4.863622,62,애월읍,"제주특별자치도 제주시 애월읍 하귀11길 39, 1층"
6,낮갈치밤돼지,4.914773,4.840024,88,애월읍,"제주특별자치도 제주시 애월읍 신엄7길 9, 1층"
7,와썹,4.933824,4.837649,68,애월읍,"제주특별자치도 제주시 애월읍 곽지1길 30, 1층 제1호"
8,금돈가,4.857488,4.826738,207,애월읍,"제주특별자치도 제주시 애월읍 가문동남길 63, 1층"
9,애월포구횟집,4.875000,4.815521,104,애월읍,"제주특별자치도 제주시 애월읍 애월로11길 22, 2층"


In [12]:
 make_review_data(data, input_data_area)

,rating,weighted_rating,num,category,address
name,,,,,
쉬다가게,4.960648,4.923976,216,애월읍,"제주특별자치도 제주시 애월읍 구엄길 66, 1층"
호커센터,4.920394,4.906717,559,애월읍,제주특별자치도 제주시 애월읍 애월로11길 25-2
제주기와,4.913000,4.897989,500,애월읍,"제주특별자치도 제주시 애월읍 광령남4길 45-1, 1층"
포구횟집,4.898798,4.884199,499,애월읍,제주특별자치도 제주시 애월읍 가문동길 43
누렁소앤도새기,4.948980,4.875987,98,애월읍,"제주특별자치도 제주시 애월읍 광령남6길 14, 1층"
카페앤틱애월,4.975806,4.863622,62,애월읍,"제주특별자치도 제주시 애월읍 하귀11길 39, 1층"
낮갈치밤돼지,4.914773,4.840024,88,애월읍,"제주특별자치도 제주시 애월읍 신엄7길 9, 1층"
와썹,4.933824,4.837649,68,애월읍,"제주특별자치도 제주시 애월읍 곽지1길 30, 1층 제1호"
금돈가,4.857488,4.826738,207,애월읍,"제주특별자치도 제주시 애월읍 가문동남길 63, 1층"


In [13]:
 make_review_data(data, '서귀포시')

,rating,weighted_rating,num,category,address
name,,,,,
성산흑돈쭐,4.972973,4.941318,259,서귀포시,제주특별자치도 서귀포시 성산읍 성산중앙로 72
대한민육,4.959834,4.937301,361,서귀포시,제주특별자치도 서귀포시 서문로 30 (서귀동)
흑돼지해물삼합,4.927785,4.922947,1634,서귀포시,제주특별자치도 서귀포시 태평로482번길 50 (서귀동)
중문색달통갈치,4.902311,4.894486,952,서귀포시,"제주특별자치도 서귀포시 일주서로 993, 1층 101호 (색달동)"
카페블루하우스,4.895620,4.888763,1073,서귀포시,"제주특별자치도 서귀포시 중앙로 73, 1층 (서귀동)"
중문흑돼지천국,4.890385,4.881109,780,서귀포시,"제주특별자치도 서귀포시 이어도로 137, 1층 (대포동)"
만족한상회,4.890299,4.879537,670,서귀포시,"제주특별자치도 서귀포시 중문상로 58-5, 1층 (중문동)"
칼초네화덕피자,4.887967,4.878028,723,서귀포시,제주특별자치도 서귀포시 예래로 115 (상예동)
포트그릴스테이크하우스,4.898268,4.867864,231,서귀포시,"제주특별자치도 서귀포시 솔동산로10번길 48, 1층 (서귀동)"


## 맛집 추천 시스템(컨텐츠 기반)

In [ ]:
def make_relation(data):
    
    # 사용자 별 식당 별점 피봇테이블 -  개오래걸림
    piv = data.pivot_table(values='rating',index='write_id',columns='name')
    
    # NaN 값을 모두 0 으로 변환 - 오래걸림
    piv = piv.fillna(0)
    
    # 사용자-아이템 행렬 데이터를 아이템-사용자 행렬 데이터로 변환
    piv_T = piv.transpose()
    
    # 아이템간의 코사인 유사도로 아이템 유사도 산출 - 좀 걸림
    from sklearn.metrics.pairwise import cosine_similarity
    piv_sim = cosine_similarity(piv_T,piv_T)

    # cosine_similarity() 로 반환된 넘파이 행렬을 영화명을 매핑하여 DataFrame으로 변환
    piv_sim_df = pd.DataFrame(data=piv_sim, index=piv.columns,
                              columns=piv.columns)
    
    return piv_sim_df

In [ ]:
piv_sim_df=make_relation(data)

In [25]:
# 매번 돌리면 오래걸리기 때문에 저장 후 사용

piv_sim_df.to_csv('data/relation_food.csv')

In [30]:
data2 = pd.read_csv('data/relation_food.csv',index_col=['name'])

## 실제 구현 부분

In [ ]:
# name과 유사도가 높은 상위 10개 식당 추출

data2 = pd.read_csv('data/relation_food.csv',index_col=['name'])

In [77]:
# 해당 지점을 DB랑 연결

if __name__=='__main__':
    print('만족한 식당을 입력해 주세요')
    print()
    input_data_rest=input()
    # 관련 식당 info에 저장
    info = data2[input_data_rest].sort_values(ascending=False)[1:11]
    info = pd.DataFrame(info).merge(pd.DataFrame(data.groupby(['name'])['address'].first()),left_index=True,right_on='name',how='inner')
    print()
    print()
    
    # 관련 식당 순위 출력
    for i in range(10):
        print(i+1,'순위')
        print('음식점 이름 : ', info.index[i], '\t연관성 : ', round(info[input_data_rest][i],2))
        print()
        print()

만족한 식당을 입력해 주세요

성산흑돈쭐


1 순위
음식점 이름 :  우돈향 	연관성 :  0.44


2 순위
음식점 이름 :  기쁨이네해물탕앤돌솥밥 	연관성 :  0.38


3 순위
음식점 이름 :  애월포구횟집 	연관성 :  0.33


4 순위
음식점 이름 :  해미원횟집 	연관성 :  0.33


5 순위
음식점 이름 :  연탄과친한돼지 	연관성 :  0.33


6 순위
음식점 이름 :  만세본점 	연관성 :  0.31


7 순위
음식점 이름 :  신우성흑돼지 	연관성 :  0.28


8 순위
음식점 이름 :  제주정지 	연관성 :  0.28


9 순위
음식점 이름 :  꿀맛 	연관성 :  0.28


10 순위
음식점 이름 :  중문회포장센터새벽야시장 	연관성 :  0.28




In [32]:
# name과 유사도가 높은 상위 6개 식당 추출
data2['성산흑돈쭐'].sort_values(ascending=False)[1:7]

name
우돈향            0.437549
기쁨이네해물탕앤돌솥밥    0.379450
애월포구횟집         0.331223
해미원횟집          0.325875
연탄과친한돼지        0.325105
만세본점           0.307287
Name: 성산흑돈쭐, dtype: float64

In [35]:
aa=data2['성산흑돈쭐'].sort_values(ascending=False)[1:7]

In [69]:
pd.DataFrame(info).merge(pd.DataFrame(data.groupby(['name'])['address'].first()),left_index=True,right_on='name',how='inner')

,성산흑돈쭐,address
name,,
우돈향,0.437549,"제주특별자치도 제주시 조천읍 조함해안로 530, 2층"
기쁨이네해물탕앤돌솥밥,0.379450,제주특별자치도 제주시 조천읍 신북로 527-1
애월포구횟집,0.331223,"제주특별자치도 제주시 애월읍 애월로11길 22, 2층"
해미원횟집,0.325875,제주특별자치도 서귀포시 표선면 민속해안로 578-2
연탄과친한돼지,0.325105,"제주특별자치도 제주시 원노형로 61, 1층 (노형동)"
만세본점,0.307287,제주특별자치도 제주시 오라로 152 (오라일동)
신우성흑돼지,0.284904,"제주특별자치도 서귀포시 중문관광로72번길 29-9, 1층 (색달동)"
제주정지,0.283028,"제주특별자치도 제주시 서해안로 480, 1층 (용담삼동)"
꿀맛,0.280240,"제주특별자치도 서귀포시 막숙포로 95, 1층 (법환동)"


In [73]:
info

,해미원횟집,address
name,,
기쁨이네해물탕앤돌솥밥,0.373190,제주특별자치도 제주시 조천읍 신북로 527-1
우돈향,0.373052,"제주특별자치도 제주시 조천읍 조함해안로 530, 2층"
신우성흑돼지,0.353222,"제주특별자치도 서귀포시 중문관광로72번길 29-9, 1층 (색달동)"
중문회포장센터새벽야시장,0.338652,제주특별자치도 서귀포시 중문관광로 293 (대포동)
성산흑돈쭐,0.325875,제주특별자치도 서귀포시 성산읍 성산중앙로 72
연탄과친한돼지,0.317279,"제주특별자치도 제주시 원노형로 61, 1층 (노형동)"
제주정지,0.307003,"제주특별자치도 제주시 서해안로 480, 1층 (용담삼동)"
대한민육,0.292306,제주특별자치도 서귀포시 서문로 30 (서귀동)
돈꼬집,0.277285,제주특별자치도 서귀포시 신서로32번길 18 (강정동)


### Kdd 유사도 기반

In [6]:
star_data_10_T = star_data_10.T
star_data_20_T = star_data_20.T

In [171]:
star_data_10_T
star_data_20_T

,0,1,2,3,4,5,6,7,8,9,...,888,889,890,891,892,893,894,895,896,897
name,,,,,,,,,,,,,,,,,,,,,
09학번,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1.5커피,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
11시11분,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
17번가,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1950김프로,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
희랑김밥,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
희야네식당,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
히비,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [7]:
rest_name_10 = pd.DataFrame(star_data_10_T.index)

rest_name_10.columns=['name']

rest_name_20 = pd.DataFrame(star_data_20_T.index)

rest_name_20.columns=['name']

In [250]:
rest_name_10.loc[1]

name    09학번
Name: 1, dtype: object

In [8]:
star_data_10_T.reset_index(drop=True,inplace=True)
star_data_20_T.reset_index(drop=True,inplace=True)

In [63]:
star_data_10_T

,0,1,2,3,4,5,6,7,8,9,...,3494,3495,3496,3497,3498,3499,3500,3501,3502,3503
0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,5.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6396,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
6397,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
6398,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
6399,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [106]:
#kdd 유사도 함수 
from math import sqrt
def sim_distance(data, n1, n2):
    sum=0
    #두 사용자가 모두 간 식당를 기준으로 해야해서 i로 변수 통일(j따로 안 써줌)
    for i in data[data[n1]>=0].index:
         if data.loc[i,n2]>=0:
            sum+=pow(data.loc[i,n1]-data.loc[i,n2],2) #누적합 
    if sum==0:
        return 0
    return sqrt(1/(sum+1)) #유사도 형식으로 출력 

In [107]:
# 나와 유사도가 높은 user 매칭 함수
def top_match(data, name, rank = 5, simf = sim_distance):
    simList = []
    for i in list(data.columns):
        if name != i:
            if simf(data,name,i)!=0:
                simList.append((simf(data, name, i), i))
    simList.sort()
    simList.reverse()    
    return simList[:rank]

In [108]:
# 추천 시스템 함수
def recommendation(data, person, simf = sim_distance):
    res = top_match(data, person) # person과 유사도가 높은 user
    score_dic = {}
    sim_dic = {}
    myList = []
    for sim, name in res:
        for movie in data.loc[:, data.loc[:, person] < 0].index:
            if sim>0:
                simSum = 0
                if data.loc[movie, name] >= 0:
                    simSum += sim * data.loc[movie, name]

                    score_dic.setdefault(movie, 0) # 평점 0으로 설정 후
                    score_dic[movie] += simSum # 

                    sim_dic.setdefault(movie, 0)
                    sim_dic[movie] += sim                
    for key in score_dic:
        myList.append((score_dic[key] / sim_dic[key], key))
    myList.sort()
    myList.reverse()
    
    return myList

In [109]:
# user가 안간 식당중에서 
#추천 점수가 가장 높은 순으로 예상평점과 식당이름을 추천 (10개까지)
movieList = []
for rate, m_id in recommendation(star_data_10_T, 1):
    movieList.append((round(rate,2), rest_name_10.loc[m_id, 'name']))
movieList[:10]

[(5.0, '회포'),
 (5.0, '홍대개미'),
 (5.0, '해상'),
 (5.0, '포동정식시청점'),
 (5.0, '티바두마리치킨삼화점'),
 (5.0, '토계정'),
 (5.0, '텐저린'),
 (5.0, '타블도트'),
 (5.0, '쿠웨이트삼화점'),
 (5.0, '코바코삼화지구점')]

In [253]:
sim_distance(star_data_10_T,1,3334)

0.8944271909999159

In [112]:
top_match(star_data_10_T,1)

[(0.8944271909999159, 3334),
 (0.8944271909999159, 2908),
 (0.8944271909999159, 2693),
 (0.8944271909999159, 2580),
 (0.8944271909999159, 2437)]

In [111]:
recommendation(star_data_10_T,1)

[(5.0, 6346),
 (5.0, 6261),
 (5.0, 6146),
 (5.0, 5830),
 (5.0, 5736),
 (5.0, 5647),
 (5.0, 5646),
 (5.0, 5559),
 (5.0, 5505),
 (5.0, 5485),
 (5.0, 5399),
 (5.0, 5266),
 (5.0, 5198),
 (5.0, 5088),
 (5.0, 4785),
 (5.0, 4770),
 (5.0, 4601),
 (5.0, 4329),
 (5.0, 4328),
 (5.0, 4226),
 (5.0, 4033),
 (5.0, 3751),
 (5.0, 3581),
 (5.0, 3539),
 (5.0, 3493),
 (5.0, 3455),
 (5.0, 3386),
 (5.0, 3301),
 (5.0, 3295),
 (5.0, 3290),
 (5.0, 3108),
 (5.0, 3061),
 (5.0, 3043),
 (5.0, 2918),
 (5.0, 2744),
 (5.0, 2722),
 (5.0, 2360),
 (5.0, 2285),
 (5.0, 2266),
 (5.0, 2105),
 (5.0, 2039),
 (5.0, 1949),
 (5.0, 1827),
 (5.0, 1798),
 (5.0, 1767),
 (5.0, 1718),
 (5.0, 1581),
 (5.0, 1532),
 (5.0, 1417),
 (5.0, 1415),
 (5.0, 1149),
 (5.0, 1136),
 (5.0, 1133),
 (5.0, 985),
 (5.0, 42),
 (4.75, 1662),
 (4.5, 6260),
 (4.5, 6241),
 (4.5, 6061),
 (4.5, 6054),
 (4.5, 5863),
 (4.5, 5860),
 (4.5, 5762),
 (4.5, 5687),
 (4.5, 5574),
 (4.5, 5303),
 (4.5, 5130),
 (4.5, 5080),
 (4.5, 4886),
 (4.5, 4864),
 (4.5, 4398),
 (4.5, 4

In [72]:
star_data_10_T.loc[:, 1]

0      -1.0
1      -1.0
2      -1.0
3      -1.0
4      -1.0
       ... 
6396   -1.0
6397   -1.0
6398   -1.0
6399   -1.0
6400   -1.0
Name: 1, Length: 6401, dtype: float64

### 피어슨상관계수 기반 - 단점 계산느림

In [100]:
#피어슨 상관계수
def sim_pearson(data, n1, n2): 
    #구현
    sumX=0
    sumY=0
    sumSqX=0 # x 제곱합 
    sumSqY=0 # y 제곱합 
    sumXY=0 #XY 합
    cnt =0 #식당 갯수 
    nums=0
    den=0
    for i in data[data[n1]>=0].index:
        if  data.loc[i,n2]>=0:
            sumX+=data.loc[i,n1]
            sumY+=data.loc[i,n2]
            sumSqX+=pow(data.loc[i,n1],2)
            sumSqY+=pow(data.loc[i,n2],2)
            sumXY+=(data.loc[i,n1])*(data.loc[i,n2])
            cnt+=1
#             global nums # 전역변수 선언
#             global den # 전역변수 선언
            nums=sumXY-((sumX*sumY)/cnt)
            den= (sumSqX-(pow(sumX,2)/cnt))*(sumSqY-(pow(sumY,2)/cnt))
    return nums/sqrt(den+0.00001) # 분모=0방지

In [101]:
#나와 유사도가 높은 user와 매칭 함수
def top_match(data, name, rank = 5, simf = sim_pearson):
    simList = []
    for i in data.columns:
        if name != i:
            if simf(data, name, i) is not None:
                simList.append((simf(data, name, i), i))
    simList.sort()
    simList.reverse()
    return simList[:rank]

In [102]:
#추천 시스템 함수
def recommendation(data, person, simf = sim_pearson):
    res = top_match(data, person)
    score_dic = {}
    sim_dic = {}
    myList = []
    for sim, name in res:
        for movie in data.loc[:, data.loc[:, person] < 0].index:
            if sim>0:
                simSum = 0
                if data.loc[movie, name] >= 0:
                    simSum += sim * data.loc[movie, name]

                    score_dic.setdefault(movie, 0)
                    score_dic[movie] += simSum

                    sim_dic.setdefault(movie, 0)
                    sim_dic[movie] += sim                
    for key in score_dic:
        myList.append((score_dic[key] / sim_dic[key], key))
    myList.sort()
    myList.reverse()
    
    return myList

In [103]:
movieList = []
for rate, m_id in recommendation(star_data_20_T, 2):
    movieList.append((round(rate,2), rest_name_20.loc[m_id, 'name']))
#     if len(movieList)==10: #10 개되면 멈추기 
#         break
movieList[:10] # 10개만 추출 

[(5.0, '흑돈플러스'),
 (5.0, '포동정식시청점'),
 (5.0, '토계정'),
 (5.0, '제주초밥'),
 (5.0, '제주명가두루치기'),
 (5.0, '저지리498맛나게드시게'),
 (5.0, '우도아름다운소풍'),
 (5.0, '애월빵공장앤카페'),
 (5.0, '빽다방베이커리제주사수본점'),
 (5.0, '블랙데이')]

In [303]:
sim_pearson(star_data_20_T,1,141)

0.999996049406127

In [301]:
top_match(star_data_20_T,1)

[(0.999996049406127, 141),
 (0.9999950000374997, 600),
 (0.9999950000374997, 40),
 (0.9999950000374997, 26),
 (0.9999911112296279, 430)]

## 평가지표

### MRR

In [18]:
def make_review_data_all(data, percentile=0.6):

    cnt_review = pd.DataFrame(data.value_counts('name'))
    avg_star = data.pivot_table('rating',index='name',aggfunc='mean')

    # 평점/리뷰갯수 정보를 가지는 temp 변수 생성
    temp = avg_star.merge(cnt_review,left_index=True,right_index=True,how='left')
    temp.columns = ['rating','num']
    
    temp= temp.fillna(0)
    
    temp

    
    m = temp['num'].quantile(percentile)
    C = temp['rating'].mean()

    
    #가충치 함수를 람다함수로 처리
    temp['weighted_rating'] = temp.apply((lambda x: (x['num']/(x['num']+m)*x['rating'])+(m/(m+x['num'])*C)), axis=1)
    
    
    ## 주소 부분
    # 음식점 이름 + 주소를 기준으로 그룹화 시켜서 
    address =pd.DataFrame(data.groupby(['name'])['address'].first())
    
    # 주소 부분 병합
    temp=temp.merge(address,left_index=True,right_index=True)
    
    
    ## 지역부분
    # 음식점 이름 + 지역을 기준으로 그룹화 시켜서 
    name_area =pd.DataFrame(data.groupby(['name'])['category'].first())
    
    # 평점/리뷰갯수에 음식점 이름 + 지역 병합
    temp=temp.merge(name_area,left_index=True,right_index=True)

    # 정렬후 반환
    return temp[['rating','weighted_rating','num','category','address']].sort_values('weighted_rating', ascending=False)

In [23]:
rank_data = make_review_data_all(data)

In [104]:
movieList

[(5.0, '흑돈플러스'),
 (5.0, '포동정식시청점'),
 (5.0, '토계정'),
 (5.0, '제주초밥'),
 (5.0, '제주명가두루치기'),
 (5.0, '저지리498맛나게드시게'),
 (5.0, '우도아름다운소풍'),
 (5.0, '애월빵공장앤카페'),
 (5.0, '빽다방베이커리제주사수본점'),
 (5.0, '블랙데이'),
 (5.0, '문희영해물아구찜'),
 (5.0, '모든날이행복하기를'),
 (5.0, '루스트플레이스아이파크점'),
 (5.0, '회포'),
 (5.0, '홍성방'),
 (5.0, '홀스버드'),
 (5.0, '홀리싯커피서플라이'),
 (5.0, '해상'),
 (5.0, '피쉬앤돈'),
 (5.0, '티바두마리치킨삼화점'),
 (5.0, '테스그릭'),
 (5.0, '타블도트'),
 (5.0, '쿠키상점'),
 (5.0, '쿠웨이트삼화점'),
 (5.0, '코바코삼화지구점'),
 (5.0, '카페정관헌'),
 (5.0, '카페쏘해피'),
 (5.0, '청아돈'),
 (5.0, '짬뽕바나나'),
 (5.0, '젬마쇼콜라'),
 (5.0, '제주정낭집'),
 (5.0, '제주몽'),
 (5.0, '이삭토스트제주중앙여고점'),
 (5.0, '육갑봉'),
 (5.0, '우무'),
 (5.0, '용꽈배기삼양점'),
 (5.0, '알맞은시간'),
 (5.0, '아라오리'),
 (5.0, '씨유제주회수사거리점'),
 (5.0, '씨유제주설촌로점'),
 (5.0, '씨유제주삼화지구점'),
 (5.0, '씨유제주삼봉로점'),
 (5.0, '신이내린닭삼화점'),
 (5.0, '시월'),
 (5.0, '세븐일레븐제주납읍점'),
 (5.0, '세븐일레븐삼양중앙점'),
 (5.0, '서가앤쿡서귀포시토평점'),
 (5.0, '뻘떡낙지서귀포점'),
 (5.0, '빗소리'),
 (5.0, '불타는여고24시떡볶이중앙여고점'),
 (5.0, '보롬'),
 (5.0, '버무리떡볶이삼화점'),
 (5.0, '바삭한하루'),
 (5.0, '밍떰본점'),
 (5

In [62]:
rank_data[rank_data.index=='고찌올레']

,rating,weighted_rating,num,category,address
name,,,,,
고찌올레,4.43125,4.43086,80,서귀포시,"제주특별자치도 서귀포시 효돈순환로 217-3, 1층 (하효동)"


In [105]:
sumx2=0
num=0
for i,name in movieList:
    sumx2 += pow(i-rank_data.loc[name,'weighted_rating'],2)
    num+=1
print(sumx2/(num-2))

0.46264364737989444
